## BUDA Capstone - ElasticSearch ETL

### Install required packages

In [3]:
pip install pandas

  Using cached pandas-1.4.3-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (11.7 MB)
  Using cached numpy-1.23.1-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.1 MB)
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install elasticsearch==7.14

  Using cached elasticsearch-7.14.0-py2.py3-none-any.whl (364 kB)
Note: you may need to restart the kernel to use updated packages.


### Import required packages

In [64]:
from elasticsearch import Elasticsearch
import json
import time
import pandas as pd
from datetime import date, timedelta

### Configure connection to elasticsearch hosts 
#### Pulled from kubernetes using 'kubectl get pods -o wide'

In [16]:
# note: the IP addresses of the pods occasionally change, wondering if there is a way to pull this from kubernetes

es = Elasticsearch(hosts=["http://172.31.0.251:9200","http://172.31.3.232:9200","http://172.31.0.171:9200"])



### Begin Queries

#### Example Query

In [17]:
#res = es.get(index="logstash-2022.07.02", id='qxkG3oEByvLGDpe-MPVW', doc_type='_doc')

In [18]:
#payload = json.loads('{"query": {"match": {"kubernetes.namespace_name": {"query":"robot-shop"}}}}')

#### Example Payload

In [19]:
#res = es.count(index="logstash-2022.07.02", body=payload)
#res

### Begin Full Index Download

In [70]:
td = date.today()-timedelta(days=5)
td = td.strftime('%Y.%m.%d')

In [71]:
#date.today().strftime('%Y.%m.%d')

In [72]:
curr_index = 'logstash-'+str(td)

In [73]:
# When indexes are fixed, uncomment and remove hardcoded line
#indexes = [curr_index]
indexes = ['logstash-2022.07.12']

In [76]:
# this loop iterates over every index above, and pulls out all robot-shop related logging

#payload = json.loads('{"query": {"match": {"kubernetes.namespace_name": {"query":"robot-shop"}}}}')

for i in indexes:
    compiled = []
    start = ['00', '02','04', '06', '08', '10', '12', '14','16','18','20','22']
    end = ['02','04', '06', '08', '10', '12', '14','16','18','20','22','23']
    _date = i[9:]
    _date = str.replace(_date, '.', '-')
    for start, end in zip(start, end):
        print(start, end)
        payload = '{"query":{"bool":{"must":[{"match":{"kubernetes.namespace_name":"robot-shop"}},{"range":{"@timestamp":{"gte":"%sT%s:00:00.000000000+00:00","lt": "%sT%s:00:00.000000000+00:00"}}}]}}}' % (_date, start, _date, end)
        res = es.search(index=i, body=payload, size=10000)
        compiled.append(json.dumps(res['hits']))
        time.sleep(5)
    f = i + '_output.txt'
    pd.DataFrame(compiled).to_csv(f)
    #with open(f, 'w') as fout:
    #    json.dump(compiled, fout)
    

00 02
02 04
04 06
06 08
08 10
10 12
12 14
14 16
16 18
18 20
20 22
22 23


### Get Result Length

In [41]:
# this calculates the total documents in all of the indexes that we have stored in the list 'compiled'
# Edit the list of indexes above to include only one, and then uncomment the below code

#length = 0
#for i in compiled:
#    length = length + len(i['hits'])